<a href="https://colab.research.google.com/github/Gabbanana/Detection-Malicious-URL/blob/master/Similarity_Comparison_N_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install nltk
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
! pip install tld
from tld import get_tld, is_tld

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412 kB 7.6 MB/s 


In [ ]:
import os
from nltk import word_tokenize
import itertools
import pandas as pd

In [ ]:
from google.colab import drive

In [ ]:
# 구글 드라이브 mount
drive.mount('/content/gdrive')

# 데이터 파일 위치
colab_path = "gdrive/My Drive/AI보안 기술개발 교육 Project/Code/"

# 데이터 파일 로드 
df = pd.read_csv(colab_path + "data/phishing_site_urls.csv")
df.head()

Mounted at /content/gdrive


,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


In [ ]:
# 데이터 전처리, bad사이트 없애고 good만 남김 -> 화이트 리스트 생성
# 열 값이 good인 행(index)만 저장
df = df[df['Label'] == 'good']

In [ ]:
df.head()

,URL,Label
18231,esxcc.com/js/index.htm?us.battle.net/noghn/en/...,good
18232,wwweira¯&nvinip¿ncH¯wVö%ÆåyDaHðû/ÏyEùuË\nÓ6...,good
18233,'www.institutocgr.coo/web/media/syqvem/dk-óij...,good
18234,Yìê koãÕ»Î§DéÎl½ñ¡ââqtò¸/à; Í,good
18236,ruta89fm.com/images/AS@Vies/1i75cf7b16vc<Fd16...,good


In [ ]:
def process_tld(url):
    try:
        res = get_tld(url, as_object = True, fail_silently=False,fix_protocol=True)
        pri_domain= res.parsed_url.netloc
    except :
        pri_domain= None
    return pri_domain

In [ ]:
df['domain'] = df['URL'].apply(lambda i: process_tld(i))

In [ ]:
df['domain'] = df['domain'].replace('www.', '', regex=True)

In [ ]:
# 결측치 확인
df.isnull().sum()

URL         0
Label       0
domain    110
dtype: int64

In [ ]:
# 결측치 처리
# 열 값이 None이 아닌 행(index)만 저장
df = df.dropna()

In [ ]:
domain_df = df[['domain']]
domain_df.head(15)

,domain
18231,esxcc.com
18236,ruta89fm.com
18238,esxcc.com
18239,esxcc.com
18243,esxcc.com
18250,esxcc.com
18253,'118bm.com
18259,118bm.com
18262,esxcc.com
18267,esxcc.com


In [ ]:
df.head()

,URL,Label,domain
18231,esxcc.com/js/index.htm?us.battle.net/noghn/en/...,good,esxcc.com
18236,ruta89fm.com/images/AS@Vies/1i75cf7b16vc<Fd16...,good,ruta89fm.com
18238,esxcc.com/js/index.htm?us.battle.net/login/en/...,good,esxcc.com
18239,esxcc.com/js/index.htm?us.battle.net/login/en/...,good,esxcc.com
18243,esxcc.com/js/index.htm?us.battle.net/login/en/...,good,esxcc.com


In [ ]:
def get_plain_vocabluary():
    sentencess = [word_tokenize(domain['domain']) for index, domain in domain_df.iterrows()]
    mergesentences = list(itertools.chain.from_iterable(sentencess))
    plainvocabulary = list(set(mergesentences))
    return plainvocabulary

In [ ]:
from itertools import islice, tee

In [ ]:
def ngram_distance(s1, s2):

	def get_ngram(word, N):
		return ['#' + word[0]] + list(zip(*(islice(seq, index, None) 
			for index, seq in enumerate(tee(word, N))))) + [word[-1] + '#']

	# get n-gram for words
	ngram_s1 = get_ngram(s1, 2)
	ngram_s2 = get_ngram(s2, 2)

	# list comparison to get overlapping grams
	overlapping_grams = len(set.intersection(set(ngram_s1), set(ngram_s2)))

	# n-gram distance formula
	ngram_distance = (len(ngram_s1) + len(ngram_s2) - (2 * overlapping_grams))

	return ngram_distance

In [ ]:
def spelling_correction(sentence):
    splittedsentence = word_tokenize(sentence)
    vocwords = list(itertools.chain.from_iterable([get_plain_vocabluary()]))
    for i,word in enumerate(splittedsentence):
        if (word not in vocwords and not word.isdigit()): # ignore digits
            ngramdistances = []
            for vocword in vocwords:
                ngramdistances.append(ngram_distance(word,vocword))
            splittedsentence[i] = vocwords[ngramdistances.index(min(ngramdistances))]
        else:
            splittedsentence[i] = word
    return ' '.join(splittedsentence)

In [ ]:
print(spelling_correction("esxcduc.com"))

esxcc.com


In [ ]:
print(spelling_correction("rut48a89fm.com"))

ruta89fm.com


In [ ]:
print(spelling_correction("gogle.com"))

google.com


*단점* : 시간이 오래걸림

1분 21초 소요